# Sea Level Rise AR5
Notebook environment to migrate netcdf files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
# Import standard packages
import os
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc
import numpy.ma as ma

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11205479-coclico", "data")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

C:\Users\kras\Anaconda3\envs\bathymetry_env\lib\site-packages\xarray\backends\cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
ds_dir = coclico_data_dir.joinpath("18_AR5_SLP_IPCC")
ds_rcp26_path = ds_dir.joinpath("total-ens-slr-26-5.nc")
ds_rcp45_path = ds_dir.joinpath("total-ens-slr-45-5.nc")
ds_rcp85_path = ds_dir.joinpath("total-ens-slr-85-5.nc")
ds_out_file = "total-ens-slr-AR5.nc"
CF_dir = coclico_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [64]:
# open datasets
ds_26rcp = xr.open_dataset(ds_rcp26_path)
ds_45rcp = xr.open_dataset(ds_rcp45_path)
ds_85rcp = xr.open_dataset(ds_rcp85_path)

# check original dataset
ds_26rcp

<xarray.Dataset>
Dimensions:     (time: 94, bnds: 2, lat: 180, lon: 360, ens: 16)
Coordinates:
  * time        (time) object 2007-07-02 12:00:00 ... 2100-07-02 12:00:00
  * lat         (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon         (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Dimensions without coordinates: bnds, ens
Data variables:
    time_bnds   (time, bnds) object ...
    totslr_ens  (ens, time, lat, lon) float32 ...
    totslr      (time, lat, lon) float32 ...
    loerr       (time, lat, lon) float32 ...
    hierr       (time, lat, lon) float32 ...
    modelname   (ens) |S16 ...
Attributes:
    history:     version 5, 27-March-2014
    contact:     mark.carson@zmaw.de
    references:  IPCC AR5 WGI, Chapter 13, and Chapter 13: Supplementary Mate...

<IPython.core.display.Javascript object>

In [5]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=ds_rcp26_path, working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [6]:
# save original CF compliancy
save_compliancy(cap, testfile=ds_rcp26_path, working_dir=CF_dir)

{'P:\\11205479-coclico\\data\\18_AR5_SLP_IPCC\\total-ens-slr-26-5.nc': {'warnings': '1', 'errors': '0'}}


<IPython.core.display.Javascript object>

In [7]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=ds_rcp45_path, working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [8]:
# save original CF compliancy
save_compliancy(cap, testfile=ds_rcp45_path, working_dir=CF_dir)

{'P:\\11205479-coclico\\data\\18_AR5_SLP_IPCC\\total-ens-slr-45-5.nc': {'warnings': '5', 'errors': '4'}}


<IPython.core.display.Javascript object>

In [9]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=ds_rcp85_path, working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [10]:
# save original CF compliancy
save_compliancy(cap, testfile=ds_rcp85_path, working_dir=CF_dir)

{'P:\\11205479-coclico\\data\\18_AR5_SLP_IPCC\\total-ens-slr-85-5.nc': {'warnings': '5', 'errors': '4'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [67]:
ds_26rcp

<xarray.Dataset>
Dimensions:     (time: 94, nv: 2, lat: 180, lon: 360, nensemble: 16)
Coordinates:
  * time        (time) object 2007-07-02 12:00:00 ... 2100-07-02 12:00:00
  * lat         (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon         (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    ensemble    (nensemble) |S16 b'       GISS-E2-R' ... b'      BCC-CSM1-1'
Dimensions without coordinates: nv, nensemble
Data variables:
    time_bnds   (time, nv) object 2007-01-01 00:00:00 ... 2101-01-01 00:00:00
    totslr_ens  (nensemble, time, lat, lon) float32 ...
    totslr      (time, lat, lon) float32 ...
    loerr       (time, lat, lon) float32 ...
    hierr       (time, lat, lon) float32 ...
Attributes:
    history:      version 5, 27-March-2014
    contact:      mark.carson@zmaw.de
    references:   IPCC AR5 WGI, Chapter 13, and Chapter 13: Supplementary Mat...
    Conventions:  CF-1.8

<IPython.core.display.Javascript object>

In [65]:
# NetCDF attribute alterations

# # add global attributes
ds_26rcp.attrs["Conventions"] = "CF-1.8"
ds_45rcp.attrs["Conventions"] = "CF-1.8"
ds_85rcp.attrs["Conventions"] = "CF-1.8"

<IPython.core.display.Javascript object>

In [66]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
ds_26rcp = ds_26rcp.rename_dims(
    {"ens": "nensemble", "bnds": "nv"}  # nv = number of vertices
)
ds_45rcp = ds_45rcp.rename_dims({"ens": "nensemble", "bnds": "nv"})
ds_85rcp = ds_85rcp.rename_dims({"ens": "nensemble", "bnds": "nv"})

# # rename variables, if necessary
ds_26rcp = ds_26rcp.rename_vars({"modelname": "ensemble"})
ds_45rcp = ds_45rcp.rename_vars({"modelname": "ensemble"})
ds_85rcp = ds_85rcp.rename_vars({"modelname": "ensemble"})

# # set some data variables to coordinates to avoid duplication of dimensions in later stage
ds_26rcp = ds_26rcp.set_coords(["ensemble"])
ds_45rcp = ds_45rcp.set_coords(["ensemble"])
ds_85rcp = ds_85rcp.set_coords(["ensemble"])

# encoding settings
ds_26rcp.time_bnds.encoding[
    "_FillValue"
] = None  # xarray sets _FillValue automatically to None for float types, prevent this when needed
ds_45rcp.time_bnds.encoding[
    "_FillValue"
] = None  # xarray sets _FillValue automatically to None for float types, prevent this when needed
ds_85rcp.time_bnds.encoding[
    "_FillValue"
] = None  # xarray sets _FillValue automatically to None for float types, prevent this when needed

strip_ensembles = ['canESM2','INMCM4','CNRM-CM5','HadGEM2-CC', 'ACCESS1-0'] # remove these at correct positions in the data from RCP45 & RCP85
# TODO construct automatically by comparing ds_26rcp["ensemble"].values.astype("U").tolist() & ds_85rcp["ensemble"].values.astype("U").tolist()

# info on all attributes
# !ncinfo -v totslr {ds_rcp26_path}

<IPython.core.display.Javascript object>

In [95]:
ds_85rcp

<xarray.Dataset>
Dimensions:     (time: 94, nv: 2, lat: 180, lon: 360, nensemble: 21)
Coordinates:
  * time        (time) object 2007-07-02 12:00:00 ... 2100-07-02 12:00:00
  * lat         (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon         (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    ensemble    (nensemble) |S16 b'       GISS-E2-R' ... b'      BCC-CSM1-1'
Dimensions without coordinates: nv, nensemble
Data variables:
    time_bnds   (time, nv) object 2007-01-01 00:00:00 ... 2101-01-01 00:00:00
    totslr_ens  (nensemble, time, lat, lon) float32 ...
    totslr      (time, lat, lon) float32 ...
    loerr       (time, lat, lon) float32 ...
    hierr       (time, lat, lon) float32 ...
Attributes:
    history:      version 5, 27-March-2014
    contact:      mark.carson@zmaw.de
    references:   IPCC AR5 WGI, Chapter 13, and Chapter 13: Supplementary Mat...
    Conventions:  CF-1.8

<IPython.core.display.Javascript object>

In [94]:
# concat datasets along new dimension with index values and name derived from pandas index object, if necessary
dataset = xr.concat(
    [ds_26rcp, ds_45rcp, ds_85rcp],
    dim="nscenarios",
    data_vars="minimal",
    coords="different",
)
dataset = dataset.assign_coords(
    scenarios=("nscenarios", np.array(["RCP26", "RCP45", "RCP85"], dtype="S"))
)

# dataset = xr.concat(
#     [dataset_historical, dataset_45rcp, dataset_85rcp],
#     pd.Index(["historical", "rcp45", "rcp85"], name="scenarios"),
# )

# dataset["scenarios"].values.astype("U") # retrieve scenarios as string

ValueError: arguments without labels along dimension 'nensemble' cannot be aligned because they have different dimension sizes: {16, 21}

<IPython.core.display.Javascript object>

In [126]:
# re-order shape of the data variables
ds_hist = ds_hist.transpose("stations", "rp", "time", "nensemble")
ds_45rcp = ds_45rcp.transpose("stations", "rp", "time", "nensemble")
ds_85rcp = ds_85rcp.transpose("stations", "rp", "time", "nensemble")
dataset = dataset.transpose("nscenarios", "stations", "rp", "time", "nensemble")

# add or change certain variable / coordinate attributes
dataset_attributes = {
    "scenarios": {"long_name": "climate scenarios"}
}  # specify custom (CF convention) attributes

# add / overwrite attributes
for k, v in dataset_attributes.items():
    try:
        dataset[k].attrs = dataset_attributes[k]
    except:
        continue

<IPython.core.display.Javascript object>

In [124]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first

dataset
# dataset["scenarios"]

<xarray.Dataset>
Dimensions:    (stations: 11014, nscenarios: 2, rp: 8, nensemble: 3, time: 11)
Coordinates:
    lon        (stations) float64 -8.991 -9.493 -9.472 ... 41.55 41.68 41.72
    lat        (stations) float64 37.03 38.78 38.75 37.02 ... 41.52 41.67 41.7
  * rp         (rp) float32 5.0 10.0 20.0 50.0 100.0 200.0 500.0 1e+03
    ensemble   (nensemble) |S4 b'min' b'mean' b'max'
  * time       (time) float32 1.995e+03 2.01e+03 2.02e+03 ... 2.09e+03 2.1e+03
    scenarios  (nscenarios) |S5 b'rcp45' b'rcp85'
Dimensions without coordinates: stations, nscenarios, nensemble
Data variables:
    esl        (nscenarios, stations, rp, time, nensemble) float64 3.302 ... ...
    eewl       (nscenarios, stations, rp, time, nensemble) float64 1.535 ... ...
Attributes:
    title:            Extreme Sea Levels
    Institution:      Joint European Research Center, Directorate for Space, ...
    reference:         Vousdoukas MI., Mentaschi L., Voukouvalas E., Verlaan ...
    email:            michail.vousdoukas@ec.europa.eu
    version:          1.2
    terms_for_use:    European Union, 1995-2015.\nReuse is authorised, provid...
    disclaimer:       Unless the following would not be permitted or valid un...
    Project_Name:     Prototype of a first Global Integrated Coastal Impact-b...
    Project_Acronym:  CoastAlRisk
    Conventions:      CF-1.8
    Starting_date:    01-Dec-2009
    End_date:         30-Nov-2099 21:00:00

<IPython.core.display.Javascript object>

In [15]:
# save new .nc files
ds_26rcp.to_netcdf(path=str(ds_rcp26_path).replace(".nc", "_CF.nc"))
ds_45rcp.to_netcdf(path=str(ds_rcp45_path).replace(".nc", "_CF.nc"))
ds_85rcp.to_netcdf(path=str(ds_rcp85_path).replace(".nc", "_CF.nc"))
# dataset.to_netcdf(path=ds_dir.joinpath(ds_out_file + "_CF.nc"))

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [16]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(ds_rcp26_path).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [17]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=str(ds_rcp26_path).replace(".nc", "_CF.nc"), working_dir=CF_dir,
)

{'P:\\11205479-coclico\\data\\18_AR5_SLP_IPCC\\total-ens-slr-26-5_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

In [18]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(ds_rcp45_path).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [19]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=str(ds_rcp45_path).replace(".nc", "_CF.nc"), working_dir=CF_dir,
)

{'P:\\11205479-coclico\\data\\18_AR5_SLP_IPCC\\total-ens-slr-45-5_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

In [20]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(ds_rcp85_path).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [21]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=str(ds_rcp85_path).replace(".nc", "_CF.nc"), working_dir=CF_dir,
)

{'P:\\11205479-coclico\\data\\18_AR5_SLP_IPCC\\total-ens-slr-85-5_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

In [139]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=ds_dir.joinpath(ds_out_file + "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [140]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=ds_dir.joinpath(ds_out_file + "_CF.nc"), working_dir=CF_dir,
)

{'P:\\11205479-coclico\\data\\03_sea_level_jrc\\CoastAlRisk_Europe_ESL_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

### write data to Zarr files

In [142]:
# export to zarr in write mode (to overwrite if exists)
dataset.to_zarr(ds_dir.joinpath("%s.zarr" % ds_out_file), mode="w")

<IPython.core.display.Javascript object>